In [29]:
import jamo
import Levenshtein
import pandas as pd
import re
import numpy as np

In [8]:
df = pd.read_csv('./temp_prd_name.tsv', names=['origin_prd_name'])

In [41]:
df = df.drop_duplicates(subset=['origin_prd_name'])
df = df.reset_index(drop=True)

In [42]:
df.origin_prd_name.sample(20)

71           DPC 이유리 스페셜아우라 시즌4 레오파드쿠션 리필
35                 DPC 스킨럽 스파 클렌징 기기 특별세트
75               [HONG SHOT] 홍진영 홍샷 컨실 팩트
21    조성아 티엠 바운스업 파워 팩트 본품2+리필3+립라커1종+에코백
51                     게리쏭 마유쿠션 본품 16g 2개
4         [데싱디바] 매직프레스 슬림핏 프리미엄 에디션 ★최신상★
32                   [어리틀레이지] 아이비 커버핏 빅쿠션
79          DPC 이유리 스페셜아우라 시즌4 레오파드 쿠션 본품
48             ★최신상 홀리데이 에디션/역대급 럭셔리 디자인★
3                    ★NEW★ 차홍 올 힛 롤링 스타일러
19                  비치웨이버 컬링 아이론 빅토리아 에디션
56      [HONG SHOT] 홍진영 홍샷 파워래스팅 벨벳 틴트 세트
26            ★상품평시 리필증정★ 아이비 빅 커버쿠션 특별세트
54        [숏사이즈] 데싱디바 매직프레스 슬림핏 네일 로얄 컬렉션
58         [HONG SHOT] 홍진영 홍샷 파워래스팅 벨벳 틴트
55                DPC 이유리 루즈퓨르 슈에뜨 립스틱 세트
64            닥터루인스 최신상 시즌3 0단계 코어세럼 슬림구성
70                 퍼펙트스킨 마그네틱 커버 스틱 파운데이션
23                   NEW 탱글티저 퀵드라이 볼륨 브러쉬
81                     조성아 파워링 바운스업 팩트 리필
Name: origin_prd_name, dtype: object

In [43]:
df['prd_name'] = df.origin_prd_name.apply(lambda x: re.sub(pattern='★|\d|\[|\]|\*|ml|시즌|박스|\+', repl=" ", string = x))
df['prd_name'] = df.prd_name.apply(lambda x: re.sub(pattern='( )+', repl=" ", string = x))


In [44]:
df['prd_name_jaso'] = df.prd_name.apply(lambda x: jamo.sentence_to_jaso(x))


In [45]:
jaso_list = df['prd_name_jaso'].tolist()
sent_list = df['prd_name'].tolist()


def distance_list(row):
    jaso = row['prd_name_jaso']
    sent = row['prd_name']
    jaso_score = [Levenshtein.distance(jaso, prd) for prd in jaso_list]
    sent_score = [Levenshtein.distance(sent, prd) for prd in sent_list]
    row['jaso_score'] = [jaso_score]
    row['sent_score'] = [sent_score]
    return row   

In [46]:
df = df.apply(lambda row: distance_list(row), axis=1)


In [47]:
df.sent_score = df.sent_score.apply(lambda x: x[0])
df.jaso_score = df.jaso_score.apply(lambda x: x[0])

In [48]:
def get_min_arg_num(score):
    arr_score = np.array(score)
    min_scores = arr_score.argsort()[1:5]
    return [min_scores]


df['min_sent_idx'] = df.sent_score.apply(lambda score: get_min_arg_num(score)[0])
df['min_jaso_idx'] = df.jaso_score.apply(lambda score: get_min_arg_num(score)[0])

In [51]:
for i, v in df.sample(20).iterrows():
    sent_idx_list = v['min_sent_idx']
    print(df.loc[i, 'prd_name'])
    print(df.loc[sent_idx_list, 'prd_name'])
    print('\n\n')

 NEW 차홍 올 힛 롤링 스타일러
8      NEW 차홍 올 힛 롤링 스타일러 롤 개
23       NEW 탱글티저 퀵드라이 볼륨 브러쉬
1          차홍 올 힛 스타일러 블랙에디션 
22       NEW 데싱디바 젤스트립 엣지 컬렉션
Name: prd_name, dtype: object



차홍 올 힛 스타일러 고데기 블랙에디션 Q 
1                      차홍 올 힛 스타일러 블랙에디션 
2                   차홍 올 힛 스타일러 크리스탈 에디션 
10    차홍 올 힛 스타일러 고데기 스와로브스키 크리스탈 에디션 Q- 
17       차홍 올 힛 스타일러 고데기 스와로브스키 크리스탈 에디션 
Name: prd_name, dtype: object



 상품평시 리필증정 아이비 빅 커버쿠션 특별세트
24     어리틀레이지 아이비 빅 커버쿠션 특별세트
38            아이비 빅 커버쿠션 특별세트
25       어리틀레이지 아이비 빅 커버쿠션 본품
69       어리틀레이지 아이비 빅 커버쿠션 리필
Name: prd_name, dtype: object



 HONG SHOT 홍진영 홍샷 컨실 팩트
58        HONG SHOT 홍진영 홍샷 파워래스팅 벨벳 틴트
34        HONG SHOT 홍진영 홍샷 파워래스팅 파운데이션
28       HONG SHOT 홍진영 홍샷 파워래스팅 파운데이션 
56     HONG SHOT 홍진영 홍샷 파워래스팅 벨벳 틴트 세트
Name: prd_name, dtype: object



게리쏭 마유쿠션 본품 g 개
57               게리쏭 컴플렉스 마유크림 세트
52    게리쏭 마유쿠션 본품 개 리필 개 마유크림 g 개
72                아이비 빅 커버쿠션 슬림구성
38                아이비 빅 커버쿠션 특별세트
Name: prd_name, dtype: object



DPC 이유리 쿠션 블랙에디션 특집구성
4

In [52]:
for i, v in df.sample(20).iterrows():
    sent_idx_list = v['min_jaso_idx']
    print(df.loc[i, 'prd_name'])
    print(df.loc[sent_idx_list, 'prd_name'])
    print('\n\n')

데싱디바 글로스 젤네일 스트립 프리미엄 컬렉션
0          데싱디바 젤스트립 디어뮤즈 컬렉션
22       NEW 데싱디바 젤스트립 엣지 컬렉션
15    데싱디바 매직프레스 네일 프리미엄 슬림핏 
29         데싱디바 슬림핏 프리미엄 에디션 
Name: prd_name, dtype: object



 HONG SHOT 홍진영 홍샷 파워래스팅 벨벳 틴트
56     HONG SHOT 홍진영 홍샷 파워래스팅 벨벳 틴트 세트
34        HONG SHOT 홍진영 홍샷 파워래스팅 파운데이션
28       HONG SHOT 홍진영 홍샷 파워래스팅 파운데이션 
75              HONG SHOT 홍진영 홍샷 컨실 팩트
Name: prd_name, dtype: object



 퍼펙트스킨X김우리 마그네틱 커버 스틱 파운데이션
84           퍼펙트스킨 마그네틱 커버 스틱 파운데이션
70           퍼펙트스킨 마그네틱 커버 스틱 파운데이션
82        퍼펙트스킨 마그네틱 커버 스틱 파운데이션 단품
34     HONG SHOT 홍진영 홍샷 파워래스팅 파운데이션
Name: prd_name, dtype: object



 어리틀레이지 아이비 빅 커버쿠션 본품
69       어리틀레이지 아이비 빅 커버쿠션 리필
24     어리틀레이지 아이비 빅 커버쿠션 특별세트
32         어리틀레이지 아이비 커버핏 빅쿠션
43       어리틀레이지 아이비 솔리드 퍼퓸 스틱
Name: prd_name, dtype: object



DPC 이유리 스페셜 아우라쿠션 본품 리필 체험 세트
66        DPC 이유리 스페셜 아우라 쿠션 
40    DPC 이유리 스페셜아우라 쿠션 최초공개!
65     DPC 이유리 스페셜아우라 핑크쿠션 본품
47     DPC 이유리 스페셜아우라 핑크쿠션 리필
Name: prd_name, dtype: object



게리쏭 마유쿠션 본품 개 리필 개 마유크림 g 개
51         